# Assignment

-------------

**1) Complete this notebook and make a pull request:** 

Answer questions (Q) in the space provided (A) in this notebook. When finished, copy your notebook to the `Assignment/` directory and name it `nb-6.5-<Github-username>.ipynb`. Then make a pull request to the upstream repo. The entered answers in this notebook will be simply Markdown text where I want you to interpret and describe a block of code to better understand what it is doing. Much of this code you will have seen already. 


**2) Write an importable Python package, save as a repo, and test it here.**

The package should be written as we did in our last lession (`.py` files in a directory with a setup.py file so it can be installed with `pip`). Follow instructions at the end of this notebook for how to write your package. Test it here by importing the package and executing the code at the end. It should work and give correct answers, if not, continue working on it. When you have it completed save your package as a new Github repo named `seqlib`.

### The `seqlib` package

Together we are going to write several functions here that will make up your new package called `seqlib`. It will be your job to copy these functions, organize them into a Class, save the code into a `.py` file (you can use SublimeText if you're comfortable with it for much of this, or any text editor including the one in jupyter), package the files so they can be imported as a library, and test the package so that it accomplishes the tasks which are defined at the end of this notebook. First things first, though, let's write the functions. 

In [6]:
import numpy as np
import pandas as pd

### Q.  Describe what the `mutate()` function below does:


A. `mutate()` takes any nucleobase of `"ACTG"` and changes it to one of the other nucleobases in that list (i.e. `mutate('A')` can only return `'C'`, `'T'`, or `'G'` ).

In [7]:
def mutate(base):
    "Mutate a nucleobase to any different nucleobase"
    diff = set("ACTG") - set(base)
    return np.random.choice(list(diff))

In [8]:
# test it
mutate("A")

'G'

### Q. Describe how the `simulate()` function below works:
Annotate the code by inserting lines with comments as you read through the function to make sense of it. What is being created at each step and how is it used?


A. The code creates a random sequence of length nsites and replicates it ninds times. Each base has a 10% chance of being mutated from the original base (in an identical way for any one position). The code also randomly drops bases (as though they were missing) with a 10% probability for any one base.

In [9]:
def simulate(ninds, nsites):
    """
    Simulates ninds DNA sequences of length nsites, with mutations and missing reads (probability 0.1)
    
    Args:
        ninds: the number of replicate sequences to generate and mutate
        nsites: the length of the original and replicate sequences
        
    Returns:
        A numpy array of dimensions (ninds, nsites) containing the altered sequences
    """
    # generate a random sequence of length nsites
    oseq = np.random.choice(list("ACGT"), size=nsites)
    
    # initialize an array with oseq repeated ninds times
    arr = np.array([oseq for i in range(ninds)])
    
    # create a random mask for entire array - probability of mutation for any site is 0.1
    muts = np.random.binomial(1, 0.1, (ninds, nsites))
    
    # mutate sequences
    for col in range(nsites):
        # pick a mutation
        newbase = mutate(arr[0, col])
        
        # create mask for col from muts
        mask = muts[:, col].astype(bool)
        
        # mutate randomly selected rows
        arr[:, col][mask] = newbase
    
    # create a random mask for missing bases and modify arr
    missing = np.random.binomial(1, 0.1, (ninds, nsites))
    arr[missing.astype(bool)] = "N"
    
    return arr

In [10]:
seqs = simulate(6, 15)
print(seqs)

[['C' 'A' 'A' 'N' 'T' 'G' 'N' 'N' 'T' 'T' 'G' 'T' 'C' 'C' 'T']
 ['C' 'A' 'T' 'T' 'C' 'C' 'G' 'A' 'T' 'T' 'G' 'T' 'C' 'G' 'T']
 ['C' 'A' 'A' 'T' 'T' 'C' 'G' 'A' 'T' 'T' 'G' 'T' 'N' 'C' 'T']
 ['C' 'A' 'A' 'T' 'N' 'C' 'T' 'A' 'A' 'N' 'G' 'T' 'C' 'C' 'T']
 ['C' 'A' 'A' 'T' 'T' 'N' 'T' 'A' 'T' 'T' 'G' 'T' 'C' 'C' 'N']
 ['C' 'A' 'A' 'T' 'C' 'C' 'T' 'A' 'N' 'T' 'G' 'T' 'N' 'C' 'T']]


### **Q: Describe how the `filter_missing` function works:**
Annotate the code by inserting lines with comments as you read through the function to make sense of it. How does it find columns with missing (N) values in them? How might you mprove it?

A. `filter_missing` creates a mask for elements of `arr` which have value `"N"`, then sums along the vertical (`0`) axis and divides by the length of that axis. It might be improved by changing the condition of the mask to `arr not in "ACGT"` to account for possible situations in which missing reads are not coded by `"N"`. The function could also use the `.copy()` function to return a copy of `arr[:, freqmissing <= maxfreq]` rather than a view of part of that array.

In [11]:
def filter_missing(arr, maxfreq):
    """
    Function to remove sites with a high frequency of missing reads from an array of DNA seqs
    
    Args:
        arr: The array of DNA sequences, with missing reads designated as N
        maxfreq: The critical frequency for missing reads. If the frequency of N at a site 
                 exceeds maxfreq, that site is excluded from the result
    
    Returns:
        A view of arr including only columns in which the frequency of N < maxfreq
    """
    # determine the frequency of missing reads at a site
    freqmissing = np.sum(arr == "N", axis=0) / arr.shape[0]
    
    # return arr, sliced to only include cols with freqmissing < maxfreq
    return arr[:, freqmissing <= maxfreq]

In [12]:
filter_missing(seqs, 0.1)

array([['C', 'A', 'A', 'G', 'T', 'C'],
       ['C', 'A', 'T', 'G', 'T', 'G'],
       ['C', 'A', 'A', 'G', 'T', 'C'],
       ['C', 'A', 'A', 'G', 'T', 'C'],
       ['C', 'A', 'A', 'G', 'T', 'C'],
       ['C', 'A', 'A', 'G', 'T', 'C']], dtype='<U1')

### **Q: Describe how the `filter_maf` function works:**
Annotate the code by inserting lines with comments as you read through the function to make sense of it. How does it calculate minor allele frequencies? Why does it use copy?

A. The minor allele frequencies are first calculated by determining the proportion of alleles at a site which differ from the corresponding allele in the first read. Then, if this proportion is greater than 0.5 (i.e. the allele in that site for the first read was not the major allele), it is replaced with its complement. A copy of `freqs` is used so that the original array is not modified.

In [16]:
def filter_maf(arr, minfreq):
    """
    Filters an array of seqs to only include sites at which a minimum proportion of alleles differ from the majority
    
    Args:
        arr: the array of sequence data
        minfreq: the minimum proportion of alleles which must differ from the most common allele at a site
        
    Returns:
        A view of the numpy array arr, filtered to include only columns in which the minor allele frequency exceeds minfreq
    """
    # determine the proportion of alleles in a col that do not match the first row
    freqs = np.sum(arr != arr[0], axis=0) / arr.shape[0]
    
    # create a copy of freqs so that all values > 0.5 are changed to their complement
    maf = freqs.copy()
    maf[maf > 0.5] = 1 - maf[maf > 0.5]
    
    return arr[:, maf > minfreq]

In [17]:
filter_maf(seqs, 0.1)

array([['A', 'N', 'T', 'G', 'N', 'N', 'T', 'T', 'C', 'C', 'T'],
       ['T', 'T', 'C', 'C', 'G', 'A', 'T', 'T', 'C', 'G', 'T'],
       ['A', 'T', 'T', 'C', 'G', 'A', 'T', 'T', 'N', 'C', 'T'],
       ['A', 'T', 'N', 'C', 'T', 'A', 'A', 'N', 'C', 'C', 'T'],
       ['A', 'T', 'T', 'N', 'T', 'A', 'T', 'T', 'C', 'C', 'N'],
       ['A', 'T', 'C', 'C', 'T', 'A', 'N', 'T', 'N', 'C', 'T']],
      dtype='<U1')

### Q: What order should these functions be applied, does it matter?

A. It doesn't matter what order `filter_missing` and `filter_maf` are applied in, because both operate on individual columns in an array, and if a site would be filtered out by both functions, it does not matter which function catches it first.

In [18]:
filter_missing(filter_maf(seqs, 0.1), 0.1)

array([['A', 'C'],
       ['T', 'G'],
       ['A', 'C'],
       ['A', 'C'],
       ['A', 'C'],
       ['A', 'C']], dtype='<U1')

In [19]:
filter_maf(filter_missing(seqs, 0.1), 0.1)

array([['A', 'C'],
       ['T', 'G'],
       ['A', 'C'],
       ['A', 'C'],
       ['A', 'C'],
       ['A', 'C']], dtype='<U1')

### Q: Describe how `calculate_statistics()` works


A. `calculate_statistics` returns several statistics about an array generated using `simulate`. These statistics are calculated as follows:
* _Mean nucleotide diversity:_ A mask of `arr` is generated with `True` (`== 1`) for bases that were identical to the site in the first read, and `False` (`== 0`) at all other bases. The variance of this mask is taken at all sites (along `axis=0`) and these variances are averaged to give mean diversity.
* _Mean minor allele frequency_: The frequency of alleles that differ from the first read is calculated as in `filter_maf`, although there is no step to ensure that the minor allele frequencies are always less than 0.5. These frequencies are then averaged.
* _Invariant sites:_ Using `np.any`, find a boolean array of sites which do not vary. Sum this to get an integer value for invariant sites.
* _Variant sites:_ Subtract `inv` from the number of sites in `arr` (`arr.shape[1]`)

In [25]:
def calculcate_statistics(arr):
    "Return several statistics for a numpy array of DNA sequences"
    # mean nucleotide diversity
    nd = np.var(arr == arr[0], axis=0).mean()
    # mean frequency of minor allele at each site
    mf = np.mean(np.sum(arr != arr[0], axis=0) / arr.shape[0])
    # number of sites with no mutations or missing reads
    inv = np.any(arr != arr[0], axis=0).sum()
    # number of sites with mutations and missing reads
    var = arr.shape[1] - inv
    return pd.Series(
        {"mean nucleotide diversity": nd,
         "mean minor allele frequency": mf,
         "invariant sites": inv,
         "variable sites": var,
        })

In [27]:
calculcate_statistics(seqs)

invariant sites                11.000000
mean minor allele frequency     0.344444
mean nucleotide diversity       0.120370
variable sites                  4.000000
dtype: float64

### Instructions: Write a `seqlib` Class object

I started writing the bare bones of it below. You should write it so that it can be executed as described below to perform all of the functions we defined above, and so that its attributes can be accessed. Save this class object in a `.py` file and make it into an importable package called `seqlib`. You can write and test your object in this notebook if you like, but it must be saved separately in a `.py` file and be imported. You cannot execute the code at the end using your object defined here in the notebook. When finished save your package to GitHub as a repo just like we did with the `helloworld` package. You do not need to write a CLI script like we did for the `helloworld` package, we will only be using the Python API here. See the examples below for **how you should write your Class object**. It should be able to run in the way written below, so look at that code and think about how you would write a Class object that can do that. 

While you can mostly copy the functions from above, you will need to modify them slightly to access information about the Class object using *self*. For example, the `simulate()` function below takes self as a first argument and can access `self.inds` and `self.nsites` from that, so we do not need to provide those as arguments to the `simulate` function like we did above. 

In [42]:
class Seqlib:
    def __init__(self, ninds, nsites):
        self.ninds = ninds
        self.nsites = nsites
        self.seqs = self.simulate()
        # ...
    
    def mutate(base):
        "Mutate a nucleobase to any different nucleobase"
        diff = set("ACTG") - set(base)
        return np.random.choice(list(diff))
    
    def simulate(self):
        """
        Simulates ninds DNA sequences of length nsites, with mutations and missing reads (probability 0.1)

        Returns:
            A numpy array of dimensions (ninds, nsites) containing the altered sequences
        """
        # generate a random sequence of length nsites
        oseq = np.random.choice(list("ACGT"), size=self.nsites)

        # initialize an array with oseq repeated ninds times
        arr = np.array([oseq for i in range(self.ninds)])

        # create a random mask for entire array - probability of mutation for any site is 0.1
        muts = np.random.binomial(1, 0.1, (self.ninds, self.nsites))

        # mutate sequences
        for col in range(self.nsites):
            # pick a mutation
            newbase = mutate(arr[0, col])

            # create mask for col from muts
            mask = muts[:, col].astype(bool)

            # mutate randomly selected rows
            arr[:, col][mask] = newbase

        # create a random mask for missing bases and modify arr
        missing = np.random.binomial(1, 0.1, (self.ninds, self.nsites))
        arr[missing.astype(bool)] = "N"

        return arr
        
    def filter_missing(arr, maxfreq):
        """
        Function to remove sites with a high frequency of missing reads from an array of DNA seqs

        Args:
            arr: The array of DNA sequences, with missing reads designated as N
            maxfreq: The critical frequency for missing reads. If the frequency of N at a site 
                     exceeds maxfreq, that site is excluded from the result

        Returns:
            A view of arr including only columns in which the frequency of N <= maxfreq
        """
        # determine the frequency of missing reads at a site
        freqmissing = np.sum(arr == "N", axis=0) / arr.shape[0]

        # return arr, sliced to only include cols with freqmissing <= maxfreq
        return arr[:, freqmissing <= maxfreq]

    def filter_maf(arr, minfreq):
        """
        Filters an array of seqs to only include sites at which a minimum proportion of alleles differ from the majority

        Args:
            arr: the array of sequence data
            minfreq: the minimum proportion of alleles which must differ from the most common allele at a site

        Returns:
            A view of the numpy array arr, filtered to include only columns in which the minor allele frequency exceeds minfreq
        """
        # determine the proportion of alleles in a col that do not match the first row
        freqs = np.sum(arr != arr[0], axis=0) / arr.shape[0]

        # create a copy of freqs so that all values > 0.5 are changed to their complement
        maf = freqs.copy()
        maf[maf > 0.5] = 1 - maf[maf > 0.5]

        return arr[:, maf > minfreq]
    
    def filter(self, minmaf, maxmissing):
        "Wrapper to combine filter_missing and filter_maf into one function applied to self.seq"
        return(filter_missing(filter_maf(self.seqs, minfreq=minmaf), maxfreq=maxmissing))
    
    def calculcate_statistics(self):
        "Return several statistics for a numpy array of DNA sequences"
        # mean nucleotide diversity
        nd = np.var(self.seqs == self.seqs[0], axis=0).mean()
        # mean frequency of minor allele at each site
        mf = np.mean(np.sum(self.seqs != self.seq[0], axis=0) / self.seq.shape[0])
        # number of sites with no mutations or missing reads
        inv = np.any(self.seqs != self.seqs[0], axis=0).sum()
        # number of sites with mutations and missing reads
        var = self.seq.shape[1] - inv
        return pd.Series(
            {"mean nucleotide diversity": nd,
             "mean minor allele frequency": mf,
             "invariant sites": inv,
             "variable sites": var,
            })
    
test = Seqlib(10, 20)

In [43]:
print(test.seqs)
print(test.filter(0, 1))

[['G' 'G' 'G' 'N' 'C' 'A' 'G' 'T' 'C' 'N' 'A' 'C' 'T' 'A' 'A' 'T' 'N' 'N'
  'T' 'C']
 ['G' 'G' 'G' 'A' 'N' 'A' 'G' 'T' 'C' 'G' 'A' 'C' 'T' 'A' 'A' 'A' 'A' 'G'
  'T' 'C']
 ['G' 'G' 'G' 'G' 'C' 'A' 'G' 'N' 'C' 'G' 'A' 'A' 'A' 'C' 'A' 'A' 'A' 'C'
  'T' 'T']
 ['G' 'G' 'G' 'G' 'C' 'A' 'G' 'T' 'N' 'N' 'A' 'N' 'T' 'A' 'A' 'A' 'A' 'C'
  'T' 'N']
 ['G' 'G' 'N' 'G' 'T' 'A' 'G' 'T' 'C' 'G' 'A' 'C' 'T' 'A' 'A' 'A' 'A' 'C'
  'N' 'C']
 ['G' 'G' 'G' 'G' 'C' 'A' 'G' 'T' 'C' 'C' 'A' 'C' 'T' 'A' 'A' 'A' 'A' 'C'
  'T' 'C']
 ['G' 'G' 'G' 'A' 'C' 'A' 'G' 'T' 'C' 'C' 'A' 'A' 'T' 'C' 'A' 'A' 'A' 'G'
  'T' 'C']
 ['G' 'G' 'G' 'G' 'C' 'A' 'N' 'N' 'N' 'G' 'A' 'C' 'T' 'A' 'A' 'A' 'N' 'C'
  'T' 'T']
 ['G' 'G' 'G' 'G' 'C' 'A' 'N' 'T' 'C' 'N' 'A' 'N' 'A' 'A' 'A' 'A' 'A' 'C'
  'T' 'C']
 ['G' 'G' 'G' 'G' 'N' 'G' 'G' 'T' 'C' 'C' 'A' 'C' 'T' 'C' 'A' 'N' 'A' 'C'
  'T' 'C']]
[['G' 'N' 'C' 'A' 'G' 'T' 'C' 'N' 'C' 'T' 'A' 'T' 'N' 'N' 'T' 'C']
 ['G' 'A' 'N' 'A' 'G' 'T' 'C' 'G' 'C' 'T' 'A' 'A' 'A' 'G' 'T' 'C']
 ['G' 'G' 'C' '

In [44]:
test.calculcate_statistics()

AttributeError: 'Seqlib' object has no attribute 'seq'

## Test your package
The package should be globally importable (you ran `pip install .` or `pip install -e .` to install it), and it should be able to execute the following code without error. 

In [9]:
import seqlib

In [11]:
# init a Seqlib Class object
seqs = seqlib.Seqlib(ninds=10, nsites=50)

In [ ]:
# access attributes from the object
print(seqs.ninds, seqs.nsites)

In [ ]:
# returns the MAF of the array as an array of floats
seqs.maf

In [13]:
# return a view of the filtered sequence array by applying a new function 
# called `filter()` that applies both the maf and missing filter functions
seqs.filter(minmaf=0.1, maxmissing=0.0)

NameError: name 'filter_missing' is not defined

In [12]:
# calculate statistics for an array with the results returned as a DataFrame
seqs.calculate_statistics()

AttributeError: 'Seqlib' object has no attribute 'calculate_statistics'

In [ ]:
# calculate statistics for an array after filtering it
seqs.filter(minmaf=0.1, maxmissing=0.0).calculate_statistics()